# Interaction with Emulator - Networks within Notebooks
###### <sup>By Sarah Samuel (sasamuel@cisco.com) </sup>

You can now experiment with networking features as and when you read about them. You no longer need multiple routers connected in specific topologies to try different feature configurations. The Cisco 8000 Emulator is a software simulation of Cisco hardware and go to great lengths to be true to the real hardware.  Using the pyvxr python client, the Cisco 8000 Emulator is invoked from within the notebooks in order to bring up a virtual network of routers.

In [Notebook User Interface](Using_Notebooks.ipynb), you saw that a Jupyter notebook is a mixture of runnable code and notes written in markdown. This notebook  is very useful as an interactive document because you can now do each programatic step and experiment with the simulated network. 

Let's see how this actually happens. In this notebook, we will be using a 2 router back-to-back network topology, like below.

**TOPOLOGY**

<div>
<img src="b2b_routers.png" width="700"/>
</div>


>In the cell below, the python module **b2b_routers** contains the code to set up the python enviroment, the simulated 2-router topology as depicted in the topology diagram along with the base configuration of the routers. The variable **sim** is initiailized to the pyvxr module. The **sim.init** and **sim.up** API calls initialize the 2 routers and brings up the network. For more informaiton on the router configuration and topology, refer to the **b2b_routers.py** file in this folder.

> Play the below cell to bring up the network. Please note that the bring up can be slow, sometimes taking 10 minutes or more.

In [1]:
from b2b_routers import *
sim = Vxr()
sim.no_image_copy=True

print("Simulation starting. Please wait for the Sim status message. This may take 3-10 minutes.")
try:
    sim.start(cfg)
    status = sim.status()
    print("Sim status: ", status)
except Exception as err:
    print("Sim launch failed (%s)" % str(err))

INFO:pyvxr.vxr:v1.1.0 2021-04-23 05:35 output_dir:vxr.out
INFO:pyvxr.vxr:b10e8f6b8c14:/home/vxr/notebooks/Getting-Started/How-to-use-Notebooks
INFO:pyvxr.vxr:Extracting vxr version from '/opt/cisco/vxr2/latest/setup.sh' file.
INFO:pyvxr.vxr_session:Starting a local bash session for user:vxr


3.8.8 (default, Apr 13 2021, 19:58:26) 
[GCC 7.3.0]
Simulation starting. Please wait for the Sim status message. This may take 3-10 minutes.


INFO:pyvxr.sim:Launch: sim_dir:/nobackup/vxr/pyvxr/qkaxb19q56 sim_rel:/opt/cisco/vxr2/latest
INFO:pyvxr.sim:Stopping previous simulation (if any)
INFO:pyvxr.sim:Cleaning previous simulation (if any)
INFO:pyvxr.sim:Starting vxr: 'sim --skiphomecheck -n '
INFO:pyvxr.sim:Vxr up on host localhost
INFO:pyvxr.vxr:Getting port vector files for:r1, r2
INFO:pyvxr.console:r1:wait for XR login prompt (console output captured in vxr.out/logs/console.r1.log)
INFO:pyvxr.console:r2:wait for XR login prompt (console output captured in vxr.out/logs/console.r2.log)
INFO:pyvxr.console:r2:entering new XR username 'cisco', password 'cisco123'
INFO:pyvxr.console:r2:entering XR username 'cisco', password 'cisco123'
INFO:pyvxr.bringup:r2:login successful
INFO:pyvxr.bringup:r2:wait for IOS XR RUN state
INFO:pyvxr.bringup:r2:run state RPs:1 (expected:1) LCs:0 (expected:0)
INFO:pyvxr.bringup:r2:all nodes reached IOS XR RUN state
INFO:pyvxr.console:r2:applying initial XR config (terminal width, etc)
INFO:pyvxr.co

Sim status:  {'localhost': 'running'}


> At this point, console access to the routers is available. Play the below cell and obtain the telnet and ssh connection details of the routers. You can access the simulated router consoles directly from your laptop through telnet/ssh or you can just continue playing the subsequent code-cells of this notebook and watch the notebook access the telnet console and execute commands on the router.

In [2]:
print('Consoles can be reached by:')
print('PE1:', get_telnet_cmd(sim, 'r1'), '\nPE2:', get_telnet_cmd(sim, 'r2'))
print('or better:')
print('PE1:', get_ssh_cmd(sim, 'r1'), '\nPE2:', get_ssh_cmd(sim, 'r2'))
print('The password is cisco123')

Consoles can be reached by:
PE1: telnet 172.17.0.2 38095 
PE2: telnet 172.17.0.2 44105
or better:
PE1: ssh cisco@172.17.0.2 -p60791 
PE2: ssh cisco@172.17.0.2 -p61262
The password is cisco123


> When the below cell is played the notebook accesses the telnet consoles of the routers.

In [3]:
ports = sim.ports()
loginpe1 = telnetlib.Telnet(str(ports['r1']['HostAgent']) , str(ports['r1']['serial0']))
loginpe2 = telnetlib.Telnet(str(ports['r2']['HostAgent']) , str(ports['r2']['serial0']))

> If you want to execute show commands on PE1, play the below cell. Edit the commands in red in the cell below and then play this cell once again to view the outputs of other show commands. If you want to execute commands on PE2, use **loginpe2** instead of **loginpe1**. The live output from the router's console is displayed beneath this cell. 

In [4]:
loginpe2.write(('''
show version
show ip int br | i Up
''').encode('ascii'))

line = loginpe2.read_until(b'/r/n',4)
print(line.decode())


RP/0/RP0/CPU0:PE2#show version
Fri Apr 23 05:40:00.127 UTC
Cisco IOS XR Software, Version 7.3.1 LNT
Copyright (c) 2013-2021 by Cisco Systems, Inc.

Build Information:
 Built By     : ingunawa
 Built On     : Fri Feb 26 04:56:31 UTC 2021
 Build Host   : iox-ucs-020
 Workspace    : /auto/srcarchive17/prod/7.3.1/8000/ws
 Version      : 7.3.1
 Label        : 7.3.1

cisco 8000 (VXR)
cisco 8201 (VXR) processor with 32GB of memory
PE2 uptime is 4 minutes
Cisco 8201 1RU Chassis

RP/0/RP0/CPU0:PE2#show ip int br | i Up
Fri Apr 23 05:40:00.289 UTC
Loopback0                      10.0.0.9        Up              Up       default 
FourHundredGigE0/0/0/0         10.11.11.2      Up              Up       default 
FourHundredGigE0/0/0/1         10.33.33.1      Up              Up       default 
MgmtEth0/RP0/CPU0/0            192.168.254.4   Up              Up       default 
RP/0/RP0/CPU0:PE2#


> Now, from the PE1 router, lets try to ping the interface on the remote router PE2 which has the ip address 11.11.11.2. The exclamation marks (**!**) indicate that ping is successful and the remote router PE2 has responded.

In [5]:
loginpe1.write(('''
show arp 10.11.11.2
ping 10.11.11.1
ping 10.11.11.2 
''').encode('ascii'))

line = loginpe1.read_until(b'/r/n',12)
print(line.decode())


RP/0/RP0/CPU0:PE1#show arp 10.11.11.2
Fri Apr 23 05:40:44.553 UTC
RP/0/RP0/CPU0:PE1#ping 10.11.11.1
Fri Apr 23 05:40:44.657 UTC
Type escape sequence to abort.
Sending 5, 100-byte ICMP Echos to 10.11.11.1, timeout is 2 seconds:
!!!!!
Success rate is 100 percent (5/5), round-trip min/avg/max = 1/5/17 ms
RP/0/RP0/CPU0:PE1#ping 10.11.11.2 
Fri Apr 23 05:40:45.155 UTC
Type escape sequence to abort.
Sending 5, 100-byte ICMP Echos to 10.11.11.2, timeout is 2 seconds:
!!!!!
Success rate is 100 percent (5/5), round-trip min/avg/max = 10/23/43 ms
RP/0/RP0/CPU0:PE1#


> Configuration commands can also be performed using the notebook. Let's try configuring OSPF on PE1 and PE2.

In [6]:
loginpe1.write(('''
configure
router ospf 10
 router-id 10.0.0.3
 area 0
  interface FourHundredGigE0/0/0/0
  interface FourHundredGigE0/0/0/1
  interface Loopback0
commit
root
exit
''').encode('ascii'))

line = loginpe1.read_until(b'/r/n',4)
print(line.decode())

loginpe2.write(('''
configure
router ospf 10
 router-id 10.0.0.9
 area 0
  interface FourHundredGigE0/0/0/0
  interface FourHundredGigE0/0/0/1
  interface Loopback0
commit
root
exit
''').encode('ascii'))

line = loginpe2.read_until(b'/r/n',4)
print(line.decode())


RP/0/RP0/CPU0:PE1#configure
edGigE0/0/0/0
  interface FourHundredGigE0/0/0/1
  interface Loopback0
commit
root
exit
Fri Apr 23 05:41:04.023 UTC
RP/0/RP0/CPU0:PE1(config)#router ospf 10
RP/0/RP0/CPU0:PE1(config-ospf)# router-id 10.0.0.3
RP/0/RP0/CPU0:PE1(config-ospf)# area 0
RP/0/RP0/CPU0:PE1(config-ospf-ar)#  interface FourHundredGigE0/0/0/0
RP/0/RP0/CPU0:PE1(config-ospf-ar-if)#  interface FourHundredGigE0/0/0/1
RP/0/RP0/CPU0:PE1(config-ospf-ar-if)#  interface Loopback0
RP/0/RP0/CPU0:PE1(config-ospf-ar-if)#commit
Fri Apr 23 05:41:04.657 UTC
RP/0/RP0/CPU0:PE1(config-ospf-ar-if)#root
RP/0/RP0/CPU0:PE1(config)#exit
RP/0/RP0/CPU0:PE1#

RP/0/RP0/CPU0:PE2#configure
ack0
commit
root
exit
Fri Apr 23 05:41:07.370 UTC
RP/0/RP0/CPU0:PE2(config)#router ospf 10
RP/0/RP0/CPU0:PE2(config-ospf)# router-id 10.0.0.9
RP/0/RP0/CPU0:PE2(config-ospf)# area 0
RP/0/RP0/CPU0:PE2(config-ospf-ar)#  interface FourHundredGigE0/0/0/0
RP/0/RP0/CPU0:PE2(config-ospf-ar-if)#  interface FourHundredGigE0/0/0/1
RP/0/RP0/

> Check whether OSPF is in FULL State. Play this cell again until OSPF is in FULL state. It might take about 30-40 seconds. And then look for routes learnt through OSPF. The routes learnt through OSPF have an O at the beginning of the line.

In [7]:
loginpe1.write(('''
show ip ospf nei
show ip route
''').encode('ascii'))

line = loginpe1.read_until(b'/r/n',4)
print(line.decode())


RP/0/RP0/CPU0:PE1#show ip ospf nei
Fri Apr 23 05:41:53.684 UTC

* Indicates MADJ interface
# Indicates Neighbor awaiting BFD session up

Neighbors for OSPF 10

Neighbor ID     Pri   State           Dead Time   Address         Interface
10.0.0.9        1     FULL/DR         00:00:37    10.11.11.2      FourHundredGigE0/0/0/0
    Neighbor is up for 00:00:34

Total neighbor count: 1
RP/0/RP0/CPU0:PE1#show ip route
Fri Apr 23 05:41:53.830 UTC

Codes: C - connected, S - static, R - RIP, B - BGP, (>) - Diversion path
       D - EIGRP, EX - EIGRP external, O - OSPF, IA - OSPF inter area
       N1 - OSPF NSSA external type 1, N2 - OSPF NSSA external type 2
       E1 - OSPF external type 1, E2 - OSPF external type 2, E - EGP
       i - ISIS, L1 - IS-IS level-1, L2 - IS-IS level-2
       ia - IS-IS inter area, su - IS-IS summary null, * - candidate default
       U - per-user static route, o - ODR, L - local, G  - DAGR, l - LISP
       A - access/subscriber, a - Application route
       M - mobi

> The routes learnt through OSPF can be pinged even if they are not directly connected.

In [8]:
loginpe1.write(('''
ping 10.33.33.1 
''').encode('ascii'))

line = loginpe1.read_until(b'/r/n',4)
print(line.decode())


RP/0/RP0/CPU0:PE1#ping 10.33.33.1 
Fri Apr 23 05:42:05.243 UTC
Type escape sequence to abort.
Sending 5, 100-byte ICMP Echos to 10.33.33.1, timeout is 2 seconds:
!!!!!
Success rate is 100 percent (5/5), round-trip min/avg/max = 10/16/30 ms
RP/0/RP0/CPU0:PE1#


> Anything that you would normally do in the telnet or ssh console of the router can be done from the notebook. Experiment on more show commands and configuration commands on the routers by editing the code-cells in this notebook.

> Once you have finished you can shutdown the topology and clean up.

In [9]:
sim.stop()

INFO:pyvxr.vxr:Stopping sim on host localhost (dir /nobackup/vxr/pyvxr/qkaxb19q56)
INFO:pyvxr.sim:Stopping previous simulation (if any)


In [10]:
sim.clean()

INFO:pyvxr.vxr:Cleaning sim on host localhost (dir /nobackup/vxr/pyvxr/qkaxb19q56)
INFO:pyvxr.sim:Stopping previous simulation (if any)
INFO:pyvxr.sim:Cleaning previous simulation (if any)


> Clean up the sim scratch-space - delete the sim directory 

In [11]:
shutil.rmtree(sim_dir)

We have many other capabilties that can be accessed with notebooks. Explore the other network notebooks to learn more.